In [1]:
# add source to apt
# sudo apt update
# sudo apt install ni-daqmx
import time
import datetime
import pprint
import nidaqmx
import threading
import numpy as np
import nidaqmx.stream_writers

In [2]:
rate = 10
samples_per_read = rate

samples = 0
def aquire_as_thread():
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai0")

        task.timing.cfg_samp_clk_timing(rate, sample_mode=nidaqmx.constants.AcquisitionType.CONTINUOUS)

        def callback(task_handle, every_n_samples_event_type,
                     number_of_samples, callback_data):
            #print('Every N Samples callback invoked.')
            data = task.read(number_of_samples_per_channel=samples_per_read)
            with open('nilog.txt','a') as fi:
                fi.write(str(datetime.datetime.now())+'\n'+ ''.join([f'{s:.5f} ' for s in data]) +'\n')
            global samples
            samples +=1
            return 0

        task.register_every_n_samples_acquired_into_buffer_event(
            samples_per_read, callback)

        task.start()            
        while keep_alive:
            print(f' {samples}x{samples_per_read} datapoints', end = '\r')
            time.sleep(1)
            
keep_alive = True 
threading.Thread(target=aquire_as_thread, args=()).start()


In [3]:
# to stop reading thread:
if 0:
    keep_alive = False
    print(f' {samples}x{samples_per_read} datapoints')

In [4]:
import os
def get_last_lines(file, n):
    with open(file, 'rb') as f:
        if os.path.getsize(file) > 1000:
            f.seek(-1000, 2)
        data = f.readlines()
        lines = []
        for i in range(n,0,-1):
            lines.append(data[-i].decode("utf-8"))
        lines = ' / '.join(lines).replace('\r\n', '').strip()
    return lines